# TaskBot

In [1]:
from typing import Optional, List, Union, Annotated

from datetime import date, datetime
from pydantic import BaseModel, Field

from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.runnables.config import RunnableConfig

from langchain_google_vertexai import ChatVertexAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

from promptgit import Prompt

from utils import object_to_xml

In [2]:
system = Prompt.from_text("""You are an executive assistant to {user}. 
Your aim is help user manage his/her tasklist and ensure all task completion on time.
Provide actionable recommendation on next work with reasonable assumptions of workload you don't know.
If you need more information, ask for it and store it in your memory.

Today is {date}.
The tasks are {tasks}""")

In [3]:
class Task(BaseModel):
    id: Optional[int] = None
    due: date
    description: str
    status: int = 0

class TaskList(BaseModel):
    last_update: datetime = Field(default_factory=datetime.now)
    tasks: List[Task] = Field(default = [])

    def __str__(self):
        return object_to_xml(self.model_dump()['tasks'], 'task')
    
    def add(self, due: date, description: str, status: int = 0):
        if len(self.tasks) > 0:
            id = max([t.id for t in self.tasks]) + 1
        else:
            id = 1
        self.tasks.append(Task(id = id, due = due, description = description, status = status))

In [4]:
l = TaskList()
l.add(date(2024, 10, 22), 'submit chemistry homework')
l.add(date(2024, 10, 23), 'second chapter in chemistry texbook')

In [5]:
class TaskState(BaseModel):
    user: str = None
    tasks: TaskList = None
    messages: Annotated[list, add_messages] = []

In [6]:
llm = ChatVertexAI(model='gemini-1.5-flash-002')

def initiate(state: TaskState, config: RunnableConfig):
    print(config)
    return {'messages': [
        SystemMessage(str(system).format(tasks=str(state.tasks), user=state.user, date='Oct 22, 2024')),
        HumanMessage('What are my tasks for today/')]
           }

def chatbot(state: TaskState):
    return {"messages": [llm.invoke(state.messages)]}

In [7]:
graph_builder = StateGraph(TaskState)

graph_builder.add_node("initiate", initiate)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "initiate")
graph_builder.add_edge("initiate", "chatbot")
graph_builder.add_edge("chatbot", END)

checkpointer = MemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer) #, interrupt_before=['chatbot'])

In [8]:
config = {"configurable": {"thread_id": "42"}}

In [9]:
response = graph.invoke({'user': 'high school student', 'tasks': l}, config)

{'metadata': {'thread_id': '42', 'langgraph_step': 1, 'langgraph_node': 'initiate', 'langgraph_triggers': ['start:initiate'], 'langgraph_path': ('__pregel_pull', 'initiate'), 'langgraph_checkpoint_ns': 'initiate:a138ba40-71fe-be6e-3311-bbefab1d6aa0'}, 'configurable': {'thread_id': '42', '__pregel_resuming': False, '__pregel_task_id': 'a138ba40-71fe-be6e-3311-bbefab1d6aa0', '__pregel_send': functools.partial(<function local_write at 0x74c436162020>, <built-in method extend of collections.deque object at 0x74c435f99a80>, dict_keys(['__start__', 'initiate', 'chatbot'])), '__pregel_read': functools.partial(<function local_read at 0x74c436161f80>, 1, {'v': 1, 'ts': '2024-10-29T18:42:08.523563+00:00', 'id': '1ef96258-07a6-64b2-8000-3acb273799d5', 'channel_values': {'user': 'high school student', 'tasks': TaskList(last_update=datetime.datetime(2024, 10, 29, 18, 42, 8, 459629), tasks=[Task(id=1, due=datetime.date(2024, 10, 22), description='submit chemistry homework', status=0), Task(id=2, due

In [14]:
response['messages'][-1].content

"Based on your current task list, your only task for today, October 22nd, 2024, is to submit your chemistry homework (Task ID #1).  This is due today, so it's your top priority.\n\nTo help me better manage your time and workload, could you tell me:\n\n* **How long do you estimate it will take to complete the chemistry homework?**  This will help me schedule in other activities.\n* **Do you have any other commitments or appointments today?** (e.g., extracurricular activities, part-time job, family events)  Knowing this will allow me to create a more realistic schedule.\n\nOnce I have this information, I can provide a more detailed and helpful schedule for the rest of your day.\n"

In [10]:
graph.get_state(config).values['messages']

[SystemMessage(content="You are an executive assistant to high school student. \nYour aim is help user manage his/her tasklist and ensure all task completion on time.\nProvide actionable recommendation on next work with reasonable assumptions of workload you don't know.\nIf you need more information, ask for it and store it in your memory.\n\nToday is Oct 22, 2024.\nThe tasks are <task><item><id>1</id><due>2024-10-22</due><description>submit chemistry homework</description><status>0</status></item><item><id>2</id><due>2024-10-23</due><description>second chapter in chemistry texbook</description><status>0</status></item></task>", additional_kwargs={}, response_metadata={}, id='4327877a-c68a-4aba-a13a-5eff6600cb5a'),
 HumanMessage(content='What are my tasks for today/', additional_kwargs={}, response_metadata={}, id='85eb77c7-913c-4695-b240-8ac9c1019180'),
 AIMessage(content="Based on your current task list, your only task for today, October 22nd, 2024, is to submit your chemistry homewo